# Will it be delayed?

Everyone who has flown has experienced a delayed or cancelled flight. Both airlines and airports would like to improve their on-time performance and predict when a flight will be delayed or cancelled several days in advance. You are being hired to build a model that can predict if a flight will be delayed. To learn more, you must schedule a meeting with your client (me). To schedule an appointment with your client, send an event request through Google Calendar for a 15 minute meeting. Both you and your project partner must attend the meeting. Come prepared with questions to ask your client. Remember that your client is not a data scientist and you will need to explain things in a way that is easy to understand. Make sure that your communications are efficient, thought out, and not redundant as your client might get frustrated and "fire" you (this only applies to getting information from your client, this does not necessary apply to asking for help with the actual project itself - you should continuously ask questions for getting help).

For this project you must go through most all steps in the checklist. You must write responses for all items as done in the homeworks, however sometimes the item will simply be "does not apply". Keep your progress and thoughts organized in this document and use formatting as appropriate (using markdown to add headers and sub-headers for each major part). Some changes to the checklist:

* Do not do the final part (launching the product).
* Your presentation will be done as information written in this document in a dedicated section (no slides or anything like that). It should include high-level summary of your results (including what you learned about the data, the "accuracy" of your model, what features were important, etc). It should be written for your client, not your professor or teammates. It should include the best summary plots/graphics/data points.
* The models and hyperparameters you should consider during short-listing and fine-tuning will be released at a later time (dependent on how far we get over the next two weeks).
* Data retrieval must be automatic as part of the code (so it can easily be re-run and grab the latest data). Do not commit any data to the repository.
* Your submission must include a pickled final model along with this notebook.

# Framing the Problem

1. **Define the objective in business terms:** <mark> The objective for this machine learning model is to be able to figure out whether a delay or cancellation is going to happen.<mark>
2. **How will your solution be used?** <mark> This model will be used to help notify airlines  a week in advance when a suspected delay is going to happen as a preventative measure to help make sure airline companies have higher ratings and increased profits. <mark>
3. **What are the current solutions/workarounds (if any)?** <mark> Currently this is done by humans at each airport but is not as effective due to the massive amounts of data needed <mark>
4. **How should you frame this problem?** <mark> This is a supervised classification problem since we are trying to predict whether a flight is going to run normal, be delayed or cancelled. This could be an online solution due to it being run in real time to predict future outcomes of flights. <mark>
5. **How should performance be measured? Is the performance measure aligned with the business objective?** <mark> Our objective is to be able to predict at least 25% of the flights that are going to be delayed or cancelled without falsely predicting any normal flights as going to be delayed or cancelled. This does align with out business objective of being able to predict when there is going to be a delayed or cancelled flight. <mark>
6. **What would be the minimum performance needed to reach the business objective?**  <mark> Again the minimum performance that would need to be predicted is 25% of the flights that are going to be delayed or cancelled accurately without falsely predicting that a normal flight is going to be delayed. <mark>
7. **What are comparable problems? Can you reuse (personal or readily available) experience or tools?** <mark> We can reuse our bike data as it is also a supervised classification problem. We also have our other homeworks and inclass examples to be able to work off of in terms of setting up the model. <mark>
8. **Is human expertise available?** <mark> Yes our client is has experience with flight delays and has provided us with good insight and direction on where to look into for our problem. <mark>
9. **How would you solve the problem manually?** <mark> To solve this problem manually we would need to look at all the data for what has caused delays and cancellations the most and calculate a way to see what airports get affected the most to be able to more accurately predict whether or not there is going to be a delay or cancellation. <mark>
10. **List the assumptions you (or others) have made so far. Verify assumptions if possible.** 
<mark> We have made the assumption that weather is going to play a massive role in whether there is going to be a delay or not. Also the size of the airport and number of staff is going to be important in whether an airport can even properly operate which could lead to delays. <mark>

In [10]:
#Imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 

In [ ]:
data = pd.read_parquet('combined.parquet')
columns_to_keep = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'OriginAirportID', 'Origin', 'OriginCityName', 'OriginStateName' ,'DestAirportID', 'Dest', 'DestCityName', 'DestStateName', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'ArrTime', 'ArrDelayMinutes', 'Cancelled', 'CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'AirTime', 'Flights', 'Distance']
data = data[columns_to_keep]

In [15]:
data.describe()

,Year,Month,DayofMonth,DayOfWeek,OriginAirportID,DestAirportID,DepTime,DepDelay,DepDelayMinutes,ArrTime,ArrDelayMinutes,Cancelled,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,AirTime,Flights,Distance
count,1.482571e+07,1.482571e+07,1.482571e+07,1.482571e+07,1.482571e+07,1.482571e+07,1.463667e+07,1.463630e+07,1.463630e+07,1.462553e+07,1.459227e+07,1.482571e+07,2.995616e+06,2.995616e+06,2.995616e+06,2.995616e+06,2.995616e+06,1.459227e+07,14825707.0,1.482571e+07
mean,2.023509e+03,6.586377e+00,1.577153e+01,3.983661e+00,1.265432e+04,1.265433e+04,1.332564e+03,1.235834e+01,1.565029e+01,1.460146e+03,1.557941e+01,1.327458e-02,2.489971e+01,4.090163e+00,1.300443e+01,1.314020e-01,2.884685e+01,1.118473e+02,1.0,8.066236e+02
std,4.999182e-01,3.403419e+00,8.781058e+00,2.007278e+00,1.526151e+03,1.526147e+03,5.077571e+02,5.612929e+01,5.507429e+01,5.441017e+02,5.468914e+01,1.144481e-01,7.658620e+01,3.370453e+01,3.139895e+01,3.370272e+00,6.458379e+01,6.986950e+01,0.0,5.922690e+02
min,2.023000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.013500e+04,1.013500e+04,1.000000e+00,-9.900000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,1.0,1.100000e+01
25%,2.023000e+03,4.000000e+00,8.000000e+00,2.000000e+00,1.129200e+04,1.129200e+04,9.120000e+02,-6.000000e+00,0.000000e+00,1.045000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.100000e+01,1.0,3.730000e+02
50%,2.024000e+03,7.000000e+00,1.600000e+01,4.000000e+00,1.288900e+04,1.288900e+04,1.325000e+03,-2.000000e+00,0.000000e+00,1.502000e+03,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,9.400000e+01,1.0,6.490000e+02
75%,2.024000e+03,1.000000e+01,2.300000e+01,6.000000e+00,1.402700e+04,1.402700e+04,1.746000e+03,9.000000e+00,9.000000e+00,1.916000e+03,9.000000e+00,0.000000e+00,2.200000e+01,0.000000e+00,1.700000e+01,0.000000e+00,3.300000e+01,1.410000e+02,1.0,1.045000e+03
max,2.024000e+03,1.200000e+01,3.100000e+01,7.000000e+00,1.686900e+04,1.686900e+04,2.400000e+03,5.764000e+03,5.764000e+03,2.400000e+03,5.780000e+03,1.000000e+00,5.764000e+03,2.419000e+03,2.700000e+03,1.460000e+03,3.581000e+03,1.338000e+03,1.0,5.095000e+03
